In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Your OpenAQ API Key
API_KEY = "your openAQ API key"
HEADERS = {
    "accept": "application/json",
    "X-API-Key": API_KEY
}

# List of Mumbai sensor IDs to fetch data for
sensor_ids = [6945, 6948, 6956, 6959, 6967, 6987, 7850, 8039, 11602, 11604, 12024, 12039,
             60661, 3409323, 3409328, 3409329, 3409482, 3409483, 3409486, 3409510, 3409511, 3409513, 3409514]

parameters = ["pm25", "pm10", "no2", "o3"]

# Mapping of sensor IDs to location names
sensor_location_map = {
    6945: "Kurla - MPCB",
    6948: "CSIA (T2) - MPCB",
    6956: "Powai - MPCB",
    6959: "Siddharth Nagar - IITM",
    6967: "Sion - MPCB",
    6987: "Vile Parle West - MPCB",
    7850: "Bandra - MPCB",
    8039: "Mumbai (Unknown)",
    11602: "BKC - IITM",
    11604: "Deonar - IITM",
    12024: "Andheri East - IITM",
    12039: "Bhandup West - IITM",
    60661: "Siddharth Nagar - IITM",
    3409323: "Worli - MPCB",
    3409328: "BKC - IITM",
    3409329: "Deonar - IITM",
    3409482: "BKC - MPCB",
    3409483: "Chembur - MPCB",
    3409486: "Kherwadi - MPCB",
    3409510: "Byculla - BMC",
    3409511: "Shivaji Nagar - BMC",
    3409513: "Sewri - BMC",
    3409514: "Ghatkopar - BMC"
}

# Set date range (last 30 days)
end_date = datetime.utcnow()
start_date = end_date - timedelta(days=30)
date_from = start_date.isoformat() + "Z"
date_to = end_date.isoformat() + "Z"

# Final data collection list
all_records = []

# Fetch data
for sensor_id in sensor_ids:
    for param in parameters:
        print(f"⏳ Fetching {param} data for Sensor {sensor_id}")
        page = 1
        while True:
            url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements"
            params = {
                "parameter": param,
                "date_from": date_from,
                "date_to": date_to,
                "limit": 1000,
                "page": page,
                "sort": "asc"
            }

            response = requests.get(url, headers=HEADERS, params=params)

            if response.status_code == 200:
                results = response.json().get("results", [])
                if not results:
                    break  # No more pages

                for entry in results:
                    period = entry.get("period", {})
                    datetime_from = period.get("datetimeFrom", {}).get("utc")

                    coordinates = entry.get("coordinates") or {}
                    latitude = coordinates.get("latitude")
                    longitude = coordinates.get("longitude")

                    parameter_info = entry.get("parameter") or {}
                    unit = parameter_info.get("units")

                    all_records.append({
                        "sensor_id": sensor_id,
                        "location_name": sensor_location_map.get(sensor_id, "Unknown"),
                        "parameter": param,
                        "value": entry.get("value"),
                        "unit": unit,
                        "timestamp": datetime_from,
                       # "latitude": latitude,
                       # "longitude": longitude
                    })

                page += 1
                time.sleep(0.5)
            else:
                print(f"❌ Error: Sensor {sensor_id}, Param {param}, Page {page}, Status {response.status_code}")
                break

# Convert to DataFrame
df = pd.DataFrame(all_records)

# Save to CSV
if not df.empty:
    df.to_csv("mumbai_aqi_30days_raw_with_locations.csv", index=False)
    print("✅ Data saved to mumbai_aqi_30days_raw_with_locations.csv")
    print(df.head())
else:
    print("❌ No data retrieved.")


⏳ Fetching pm25 data for Sensor 6945
⏳ Fetching pm10 data for Sensor 6945
⏳ Fetching no2 data for Sensor 6945
⏳ Fetching o3 data for Sensor 6945
⏳ Fetching pm25 data for Sensor 6948
❌ Error: Sensor 6948, Param pm25, Page 17, Status 408
⏳ Fetching pm10 data for Sensor 6948
❌ Error: Sensor 6948, Param pm10, Page 17, Status 408
⏳ Fetching no2 data for Sensor 6948
❌ Error: Sensor 6948, Param no2, Page 17, Status 408
⏳ Fetching o3 data for Sensor 6948
❌ Error: Sensor 6948, Param o3, Page 17, Status 408
⏳ Fetching pm25 data for Sensor 6956
⏳ Fetching pm10 data for Sensor 6956
⏳ Fetching no2 data for Sensor 6956
⏳ Fetching o3 data for Sensor 6956
⏳ Fetching pm25 data for Sensor 6959
❌ Error: Sensor 6959, Param pm25, Page 18, Status 408
⏳ Fetching pm10 data for Sensor 6959
❌ Error: Sensor 6959, Param pm10, Page 16, Status 408
⏳ Fetching no2 data for Sensor 6959
❌ Error: Sensor 6959, Param no2, Page 17, Status 408
⏳ Fetching o3 data for Sensor 6959
❌ Error: Sensor 6959, Param o3, Page 17, Statu

In [6]:
print(df.head())

   sensor_id     location_name parameter      value   unit  \
0       6948  CSIA (T2) - MPCB      pm25  49.917572  µg/m³   
1       6948  CSIA (T2) - MPCB      pm25  79.485313  µg/m³   
2       6948  CSIA (T2) - MPCB      pm25  58.055519  µg/m³   
3       6948  CSIA (T2) - MPCB      pm25  78.538864  µg/m³   
4       6948  CSIA (T2) - MPCB      pm25  66.501968  µg/m³   

              timestamp  
0  2016-11-21T10:00:00Z  
1  2016-11-21T12:00:00Z  
2  2016-11-22T13:00:00Z  
3  2016-11-22T15:00:00Z  
4  2016-11-22T17:00:00Z  
